In [1]:
from pomegranate import *

In [2]:
burglary = DiscreteDistribution({'T': 0.001, 'F': 0.999})
earthquake = DiscreteDistribution({'T': 0.002, 'F': 0.998})
alarm = ConditionalProbabilityTable([
    ['T', 'T', 'T', 0.95],
    ['T', 'T', 'F', 0.05],
    ['T', 'F', 'T', 0.94],
    ['T', 'F', 'F', 0.06],
    ['F', 'T', 'T', 0.29],
    ['F', 'T', 'F', 0.71],
    ['F', 'F', 'T', 0.001],
    ['F', 'F', 'F', 0.999],
], [burglary, earthquake])
johncalls = ConditionalProbabilityTable([
    ['T', 'T', 0.90],
    ['T', 'F', 0.10],
    ['F', 'T', 0.05],
    ['F', 'F', 0.95],
], [alarm])
marycalls = ConditionalProbabilityTable([
    ['T', 'T', 0.70],
    ['T', 'F', 0.30],
    ['F', 'T', 0.01],
    ['F', 'F', 0.99],
], [alarm])

In [3]:
sB = State(burglary, name='burglary')      # 0
sE = State(earthquake, name='earthquake')  # 1
sA = State(alarm, name='alarm')            # 2
sJ = State(johncalls, name='johncalls')    # 3
sM = State(marycalls, name='marycalls')    # 4

In [4]:
model = BayesianNetwork('Burglary Problem')
model.add_states(sB, sE, sA, sJ, sM)

In [5]:
model.add_transition(sB, sA)
model.add_transition(sE, sA)
model.add_transition(sA, sJ)
model.add_transition(sA, sM)
model.bake()

In [6]:
result1 = model.predict_proba({})[2].parameters[0]['T']
print('P(A) =', result1)

P(A) = 0.002516442000000935


In [7]:
result2 = model.predict_proba({'marycalls': 'F'})[3].parameters[0]['T'] * model.predict_proba({})[4].parameters[0]['F']
print('P(J&&~M) =', result2)

P(J&&~M) = 0.050054875461000355


In [8]:
result3 = model.predict_proba({'johncalls': 'T', 'marycalls': 'F'})[2].parameters[0]['T']
print('P(A|J&&~M) =', result3)

P(A|J&&~M) = 0.01357388933131146


In [9]:
result4 = model.predict_proba({'alarm': 'T'})[0].parameters[0]['T']
print('P(B|A) =', result4)

P(B|A) = 0.3735512282818995


In [10]:
result5 = model.predict_proba({'johncalls': 'T', 'marycalls': 'F'})[0].parameters[0]['T']
print('P(A|J&&~M) =', result5)

P(A|J&&~M) = 0.005129858133403528


In [11]:
result6 = (result2 * (1-result5)) / model.predict_proba({})[0].parameters[0]['F']
print('P(J&&~M|~B) =', result6)

P(J&&~M|~B) = 0.049847949000000266
